# Load data from the Sands and Mac directories into an SQLite database (for use with Datasette)

This is a slightly modified version of the [notebook used to process the Tasmanian Post Office Directories](https://glam-workbench.net/libraries-tasmania/tas-pod-add-to-datasette/) in the GLAM Workbench.

It creates an SQLite database and accompanying metadata file that can be published online using Datasette.

In [4]:
# Let's import the libraries we need.
import json
import re
from pathlib import Path
import json

from natsort import natsorted, ns
from sqlite_utils import Database
import pandas as pd

In [5]:
df = pd.read_csv("sands_and_mac_ie.csv").sort_values("Date")

In [6]:
# This the basic metadata file that will be used by Datasette
# The processing steps below will add details for each table/volume into the metadata file
# Obviously you'd modify this for a different publication!

metadata = {
    "title": "Sands & McDougall's directories of Victoria, 1860 to 1974",
    "description_html": "<p>This is an experimental interface to the <a href='https://find.slv.vic.gov.au/discovery/collectionDiscovery?vid=61SLV_INST:SLV&collectionId=81213035910007636'>Sands & MacDougall directories for Victoria from 1860 to 1974</a> which have been digitised and made available by the State Library of Victoria.</p><p>This interface searches for individual lines of text, rather than pages or articles. So it points you straight to entries of interest. Once you've found something, you can view the entry within the context of the complete page, or click back to the SLV to explore further.</p>",
    "databases": {
        "sands-mcdougalls-directories-victoria": {
            "title": "Sands & McDougall's directories of Victoria, 1860 to 1974",
            "source": "State Library of Victoria",
            "source_url": "https://find.slv.vic.gov.au/discovery/collectionDiscovery?vid=61SLV_INST:SLV&collectionId=81213035910007636",
            "tables": {},
        }
    },
}

In [7]:
# Create the database
# Change the name as apporpriate!
db = Database(Path("sands_and_mac", "sands-mcdougalls-directories-victoria.db"), recreate=True)

# Create database tables for pages and volumes
page_table = db["pages"]
vols_table = db["volumes"]

# Loop through the directories of each volume created by the harvesting process (above)
# Use natsorted so that they're processed in date order
vols = natsorted(
    [d for d in Path("sands_and_mac").glob("sa*") if d.is_dir()], alg=ns.PATH
)

for i, vol in df.iterrows():
    alma_id = vol["Record ID"]
    ie_id = vol["ie_id"]
    year = str(vol["Date"])
    print(year)
    vol_path = [v for v in Path("sands_and_mac").glob(f"sa{vol['Date']}*") if v.is_dir()][0]
    pages = natsorted(
        [p for p in Path(vol_path, "alto-json").glob("*.ndjson") if p.is_file()], alg=ns.PATH
    )
    # Add a record for this volume to the database
    vols_table.insert({"vol_id": ie_id, "year": year, "alma_id": alma_id}, pk="vol_id")

    # Update the metadata file with details of this volume
    metadata["databases"]["sands-mcdougalls-directories-victoria"]["tables"]["v" + year] = {
        "title": f"Sands & McDougall's directory of Victoria, {year}",
        "source": "State Library of Victoria",
        "source_url": f"https://viewer.slv.vic.gov.au/?entity={ie_id}&mode=browse",
        "searchmode": "raw",
    }

    # Create a table for this volume. For the PO directories I'm using the year as the table name.
    # If year isn't available, some other way of naming the table would be necessary, such as the full title.
    # Need to add the v prefix because otherwise SQLite indexing fails with unhelpful error message
    vol_table = db["v" + year]
    lines = []
    for page_num, page in enumerate(pages, start=1):
        
        # text = page.read_text()
        page_id = page.stem.split("-")[1]
        page_table.insert(
            {"page_id": page_id, "page": page_num, "vol_id": ie_id},
            pk=("page_id"),
            foreign_keys=[("vol_id", "volumes")],
        )

        # Open the text file and loop through the lines
        with page.open("r") as ndjson:
            line_num = 1
            for line in ndjson:
                data = json.loads(line)
                # Get rid of blank lines
                text = data["text"].replace("\n", "").strip()
                # If line is not blank, add details to a list of lines from this page
                if text:
                    lines.append(
                        {
                            "page": page_num,
                            "line": line_num,
                            "text": text,
                            "page_id": page_id,
                            "w": data["w"],
                            "h": data["h"],
                            "x": data["x"],
                            "y": data["y"],
                            
                        }
                    )
                    line_num += 1
    # Insert all the lines from this page into the db
    try:
        vol_table.insert_all(
            lines, pk=("page", "line"), foreign_keys=[("page_id", "pages", "page_id")]
        )
    except IndexError:
        print(lines[:100])

    # Add a full text index on the line text
    vol_table.enable_fts(["text"])
    # vol_table.optimize()

# Save the updated metadata file
with open(Path("sands_and_mac", "datasette-metadata.json"), "w") as md_file:
    json.dump(metadata, md_file, indent=4)

1860
1865
1870
1875
1880
1885
1890
1895
1900
1905
1910
1915
1920
1925
1930
1935
1940
1945
1950
1955
1960
1965
1970
1974


In [23]:

abb_data = []
for year in [1955, 1965, 1974]:
    abbrs = pd.read_csv(f"sands_mac_{year}_abbr.csv", names=["suburb", "abbr"])
    for abbr in abbrs.to_dict(orient="records"):
        abbr["year"] = year
        abb_data.append(abbr)
for year in [1905]:
    abbrs = pd.read_csv(f"sands_mac_{year}_abbr.csv", names=["abbr", "suburb"])
    for abbr in abbrs.to_dict(orient="records"):
        abbr["year"] = year
        abb_data.append(abbr)

In [24]:
db = Database(Path("sands_and_mac", "sands-mcdougalls-directories-victoria.db"))
db["abbreviations"].insert_all(abb_data)

<Table abbreviations (suburb, abbr, year)>